In [1]:
%load_ext dotenv
%dotenv ../.env

In [2]:
import tinvest
from datetime import datetime
import pytz
from notebooks.get_candles import get_candles

tz = pytz.timezone("Europe/Moscow")
periods = 3
interval=tinvest.CandleResolution.min1
companies = [
  { "bbGlobal": 'BBG000BBQCY0', "ticker": "AMD" },
  { "bbGlobal": 'BBG000BF6LY3', "ticker": "CCL" },
  { "bbGlobal": 'BBG000C5Z1S3', "ticker": "MU" },
  { "bbGlobal": 'BBG00741Y1N2', "ticker": "RIVN" },
  { "bbGlobal": 'BBG000N9MNX3', "ticker": "TSLA" },
  { "bbGlobal": 'BBG00JG0FFZ2', "ticker": "PTON" },
]
now_date = datetime(2022, 2, 16, 0, 0, 0, 0, tz)

def download():
  result = {}
  for company in companies:
    result[company['bbGlobal']] = get_candles(company, now=now_date, interval=interval, periods=periods)
  
  return result

result = download()


In [3]:
from notebooks.pmax_helpers import get_trades

trades = get_trades(companies, result)

In [4]:
import math

def round_decimals_down(number:float, decimals:int=1):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

In [5]:
# skip for now
# from pypfopt import risk_models, plotting
# import matplotlib.pyplot as plt
# import plotly.express as px

# df = DataFrame()
# for company in companies:
#     figi = company["bbGlobal"]
#     ticker = company["ticker"]

#     candles = result[figi]
#     df[ticker] = candles['Close']

# exp_cov = risk_models.exp_cov(df)
# plotting.plot_covariance(exp_cov, plot_correlation=True)
# plt.show()


In [7]:
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objects as go
from functools import reduce

matplotlib.rcParams.update({'font.size': 22})

def frequences(items, ticker):
  more1 = 0
  more04 = 0
  more06 = 0
  more08 = 0

  for item in items:
    if item['run_up'] >= 1:
      more1 += 1
    
    if item['run_up'] >= 0.4:
      more04 += 1

    if item['run_up'] >= 0.6:
      more06 += 1
    
    if item['run_up'] >= 0.8:
      more08 += 1
  
  percent1 = more04 / len(items) * 100 if len(items) > 0 else 0
  percent2 = more06 / len(items) * 100 if len(items) > 0 else 0
  percent3 = more08 / len(items) * 100 if len(items) > 0 else 0
  percent4 = more1 / len(items) * 100 if len(items) > 0 else 0

  less04_list = list(filter(lambda x: x['run_up'] < 0.4, items))
  less04_profit = reduce(lambda a, b: a + b['profit'], less04_list, 0)
  
  less06_list = list(filter(lambda x: x['run_up'] < 0.6, items))
  less06_profit = reduce(lambda a, b: a + b['profit'], less06_list, 0)
  
  less08_list = list(filter(lambda x: x['run_up'] < 0.8, items))
  less08_profit = reduce(lambda a, b: a + b['profit'], less08_list, 0)
  
  less1_list = list(filter(lambda x: x['run_up'] < 1, items))
  less1_profit = reduce(lambda a, b: a + b['profit'], less1_list, 0)

  average_less04_profit = round_decimals_down(less04_profit / len(less04_list), 2) if len(less04_list) > 0 else 0
  average_less06_profit = round_decimals_down(less06_profit / len(less06_list), 2) if len(less06_list) > 0 else 0
  average_less08_profit = round_decimals_down(less08_profit / len(less08_list), 2) if len(less08_list) > 0 else 0
  average_less1_profit = round_decimals_down(less1_profit / len(less1_list), 2) if len(less1_list) > 0 else 0

  print(f"""
    ticker: {ticker}
    trades: {len(items)}
    more 0.4% : {round_decimals_down(percent1, 2)}%
    if less than 0.4% then average profit is {average_less04_profit}%. {len(less04_list)} trades
    profit: {round_decimals_down((len(items) - len(less04_list)) * 0.4, 2)}% loss: {round_decimals_down(len(less04_list) * average_less04_profit, 2)}%

    more 0.6% : {round_decimals_down(percent2, 2)}%
    if less than 0.6% then average profit is {average_less06_profit}%. {len(less06_list)} trades
    profit: {round_decimals_down((len(items) - len(less06_list)) * 0.6, 2)}% loss: {round_decimals_down(len(less06_list) * average_less06_profit, 2)}%

    more 0.8% : {round_decimals_down(percent3, 2)}%
    if less than 0.8% then average profit is {average_less08_profit}%. {len(less08_list)} trades
    profit: {round_decimals_down((len(items) - len(less08_list)) * 0.8, 2)}% loss: {round_decimals_down(len(less08_list) * average_less08_profit, 2)}%

    more 1% : {round_decimals_down(percent4, 2)}%
    if less than 1% then average profit is {average_less1_profit}%. {len(less1_list)} trades
    profit: {round_decimals_down((len(items) - len(less1_list)), 2)}% loss: {round_decimals_down(len(less1_list) * average_less1_profit, 2)}%
  """)

print(f"""
periods = {periods}
interval = {interval}
""")

def get_stats(items):
  frequences(items, ticker)

  x = list(map(lambda q: q['time'].format('DD/MM HH:mm'), items))

  fig = go.Figure()
  fig.add_trace(
    go.Bar(
      x=x,
      y=list(map(lambda q: q['run_up'], items)),
      name='run up',
      marker_color='green'
    )
  )
  fig.add_trace(
    go.Bar(
      x=x,
      y=list(map(lambda q: q['profit'], items)),
      name='profit',
      marker_color='gray'
    )
  )
  fig.update_layout(
    barmode='group',
  )

  fig.show()

for company in companies:
  figi = company["bbGlobal"]
  ticker = company["ticker"]

  filtered = list(filter(
    lambda x:
      x['time'].weekday() != 5 and x['time'].weekday() != 6 and
      ((x['time'].hour == 17 and x['time'].minute >= 30) or x['time'].hour >= 18) and
      x['time'].hour <= 19
    , trades[figi]
  ))
  print("Long and Short")
  get_stats(filtered)
  
  # short = list(filter(lambda x: x['type'] == 'short', filtered))
  # print("Short")
  # get_stats(short)
  
  # long = list(filter(lambda x: x['type'] == 'long', filtered))
  # print("Long")
  # get_stats(long)





periods = 3
interval = 1min

Long and Short

    ticker: AMD
    trades: 3
    more 0.4% : 66.66%
    if less than 0.4% then average profit is -1.5%. 1 trades
    profit: 0.8% loss: -1.5%

    more 0.6% : 66.66%
    if less than 0.6% then average profit is -1.5%. 1 trades
    profit: 1.2% loss: -1.5%

    more 0.8% : 33.33%
    if less than 0.8% then average profit is -0.99%. 2 trades
    profit: 0.8% loss: -1.98%

    more 1% : 33.33%
    if less than 1% then average profit is -0.99%. 2 trades
    profit: 1.0% loss: -1.98%
  


Long and Short

    ticker: CCL
    trades: 2
    more 0.4% : 100.0%
    if less than 0.4% then average profit is 0%. 0 trades
    profit: 0.8% loss: 0.0%

    more 0.6% : 100.0%
    if less than 0.6% then average profit is 0%. 0 trades
    profit: 1.2% loss: 0.0%

    more 0.8% : 100.0%
    if less than 0.8% then average profit is 0%. 0 trades
    profit: 1.6% loss: 0.0%

    more 1% : 100.0%
    if less than 1% then average profit is 0%. 0 trades
    profit: 2.0% loss: 0.0%
  


Long and Short

    ticker: MU
    trades: 4
    more 0.4% : 50.0%
    if less than 0.4% then average profit is -1.06%. 2 trades
    profit: 0.8% loss: -2.12%

    more 0.6% : 50.0%
    if less than 0.6% then average profit is -1.06%. 2 trades
    profit: 1.2% loss: -2.12%

    more 0.8% : 25.0%
    if less than 0.8% then average profit is -1.2%. 3 trades
    profit: 0.8% loss: -3.6%

    more 1% : 25.0%
    if less than 1% then average profit is -1.2%. 3 trades
    profit: 1.0% loss: -3.6%
  


Long and Short

    ticker: RIVN
    trades: 4
    more 0.4% : 100.0%
    if less than 0.4% then average profit is 0%. 0 trades
    profit: 1.6% loss: 0.0%

    more 0.6% : 50.0%
    if less than 0.6% then average profit is -2.22%. 2 trades
    profit: 1.2% loss: -4.45%

    more 0.8% : 50.0%
    if less than 0.8% then average profit is -2.22%. 2 trades
    profit: 1.6% loss: -4.45%

    more 1% : 50.0%
    if less than 1% then average profit is -2.22%. 2 trades
    profit: 2.0% loss: -4.45%
  


Long and Short

    ticker: TSLA
    trades: 1
    more 0.4% : 100.0%
    if less than 0.4% then average profit is 0%. 0 trades
    profit: 0.4% loss: 0.0%

    more 0.6% : 100.0%
    if less than 0.6% then average profit is 0%. 0 trades
    profit: 0.6% loss: 0.0%

    more 0.8% : 100.0%
    if less than 0.8% then average profit is 0%. 0 trades
    profit: 0.8% loss: 0.0%

    more 1% : 100.0%
    if less than 1% then average profit is 0%. 0 trades
    profit: 1.0% loss: 0.0%
  


Long and Short

    ticker: PTON
    trades: 6
    more 0.4% : 83.33%
    if less than 0.4% then average profit is -1.61%. 1 trades
    profit: 2.0% loss: -1.61%

    more 0.6% : 66.66%
    if less than 0.6% then average profit is -1.44%. 2 trades
    profit: 2.4% loss: -2.88%

    more 0.8% : 66.66%
    if less than 0.8% then average profit is -1.44%. 2 trades
    profit: 3.2% loss: -2.88%

    more 1% : 66.66%
    if less than 1% then average profit is -1.44%. 2 trades
    profit: 4.0% loss: -2.88%
  
